In [13]:
import os
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import yaml

In [15]:
def create_folder_structure(base_path):
    folders = [
        'data/train/images', 'data/train/labels',
        'data/val/images', 'data/val/labels',
        'data/test/images', 'data/test/labels'
    ]
    for folder in folders:
        os.makedirs(os.path.join(base_path, folder), exist_ok=True)

def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    return (x * dw, y * dh, w * dw, h * dh)

def convert_annotation(xml_path, output_path, classes):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(output_path, 'w') as out_file:
        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult) == 1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('ymin').text),
                 float(xmlbox.find('xmax').text), float(xmlbox.find('ymax').text))
            bb = convert_bbox((w, h), b)
            out_file.write(f"{cls_id} {' '.join(map(str, bb))}\n")

def process_images(image_files, image_folder, annotation_folder, output_path, split, classes):
    for file in image_files:
        # Image
        src_img = os.path.join(image_folder, file)
        dst_img = os.path.join(output_path, f'data/{split}/images', file)
        shutil.copy(src_img, dst_img)

        # Annotation
        xml_file = os.path.splitext(file)[0] + '.xml'
        src_xml = os.path.join(annotation_folder, xml_file)
        dst_txt = os.path.join(output_path, f'data/{split}/labels', os.path.splitext(file)[0] + '.txt')
        convert_annotation(src_xml, dst_txt, classes)

def split_and_process_dataset(image_files, image_folder, annotation_folder, output_path, classes):
    train_val, test = train_test_split(image_files, test_size=0.2, random_state=42)
    train, val = train_test_split(train_val, test_size=0.2, random_state=42)

    for split, files in [('train', train), ('val', val), ('test', test)]:
        process_images(files, image_folder, annotation_folder, output_path, split, classes)
def create_yaml(output_path, classes):
    yaml_content = {
        'train': f'{HOME}/data/train/images',
        'val': f'{HOME}/data/val/images',
        'test': f'{HOME}/data/test/images',
        'nc': len(classes),
        'names': classes
    }

    with open(os.path.join(output_path, 'data', 'data.yaml'), 'w') as yaml_file:
        yaml.dump(yaml_content, yaml_file, default_flow_style=False)        

In [17]:
if __name__ == "__main__":
    HOME = os.getcwd()
    print(HOME)
    
    create_folder_structure(HOME)

    dataset_path = '/Users/tanishapriya/Downloads/archive'
    output_path = f'{HOME}/'
    classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']
    
    image_folder = os.path.join(dataset_path, 'images')
    annotation_folder = os.path.join(dataset_path, 'annotations')
    
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    
    split_and_process_dataset(image_files, image_folder, annotation_folder, output_path, classes)
    create_yaml(output_path, classes)


/Users/tanishapriya/Desktop/Computer_Vision


In [18]:
!pip install -U ultralytics



In [1]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')  

# Train the model
res = model.train(
    data='/Users/tanishapriya/Desktop/Computer_Vision/data/data.yaml',
    epochs=10,     # Set the number of epochs 
    batch=32,       # Set the batch size to 32
    plots=True,   
    imgsz=640,
    )

Ultralytics 8.3.15 🚀 Python-3.11.0 torch-2.4.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/Users/tanishapriya/Desktop/Computer_Vision/data/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning /Users/tanishapriya/Desktop/Computer_Vision/data/train/labels.cache... 545 images, 242 backgrounds, 0 corrupt: 100%|██████████| 787/787 [00:00<?, ?it/s]
val: Scanning /Users/tanishapriya/Desktop/Computer_Vision/data/val/labels.cache... 137 images, 65 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train4
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.919      4.048      1.568         97        640:  80%|████████  | 20/25 [32:13<04:18, 51.78s/it]   libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
       1/10         0G      1.812      3.697      1.486         57        640: 100%|██████████| 25/25 [35:11<00:00, 84.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:06<00:00, 16.75s/it]

                   all        202        579    0.00262      0.263     0.0629     0.0255


...

      10/10         0G      1.074      1.143     0.9756         91        640:  84%|████████▍ | 21/25 [17:58<03:16, 49.21s/it]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
      10/10         0G      1.065      1.138     0.9686         44        640: 100%|██████████| 25/25 [21:00<00:00, 50.43s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:14<00:00, 18.66s/it]

                   all        202        579      0.709      0.525      0.383      0.255



10 epochs completed in 3.799 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 19.9MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 19.9MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.11.0 torch-2.4.1 CPU (Apple M1)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:06<00:00, 16.66s/it]


                   all        202        579       0.71      0.527      0.383      0.255
             with_mask        122        419      0.586      0.901      0.577      0.412
          without_mask         47        145      0.546      0.679      0.493      0.303
 mask_weared_incorrect         14         15          1          0     0.0804     0.0485
Speed: 0.8ms preprocess, 315.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train4


In [2]:
pip install opencv-python-headless matplotlib numpy supervision ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 3.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Evaluate model on validation/test data
results = model.val(
    data='/Users/tanishapriya/Desktop/Computer_Vision/data/data.yaml',
    batch=32,     # Set the batch size to 32
    imgsz=640,    # Image size
)


Ultralytics 8.3.15 🚀 Python-3.11.0 torch-2.4.1 CPU (Apple M1)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs


val: Scanning /Users/tanishapriya/Desktop/Computer_Vision/data/val/labels.cache... 137 images, 65 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:57<00:00,  8.20s/it]


                   all        202        579      0.709      0.528      0.382      0.253
             with_mask        122        419      0.584      0.902      0.578      0.412
          without_mask         47        145      0.542      0.683      0.492      0.301
 mask_weared_incorrect         14         15          1          0     0.0747     0.0458
Speed: 1.3ms preprocess, 269.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train42
